Extract all links:

In [2]:
import requests
from bs4 import BeautifulSoup

url = 'https://zenodo.org/record/7515352#.ZFFS_y8RppQ'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')

urls = []
for link in soup.find_all('a'):
	urls.append(link.get('href'))

In [3]:
# Extract all tifs
tifs = []
i = 0
for url in urls:
    try:
        if 'tif' in url:
            tifs.append(url)
    except:
        continue


In [4]:
# Convert into links:
links_5cm = []
links_1m = []
for tif in tifs:
    if '_005.tif' in tif:
        links_5cm.append(tif.replace('/record/','https://zenodo.org/record/'))
    else:
        links_1m.append(tif.replace('/record/','https://zenodo.org/record/'))
# Crude way of removing duplicates
links_5cm = list(set(links_5cm))
links_1m = list(set(links_1m))

In [5]:
def create_file_name(link):
    start = link.find('NWNKern_') + len('NWNKern_')
    end = link.find('.tif') + len('.tif')
    return link[start:end]

In [6]:
from osgeo import gdal

def save_img(link):
    print('opening image')
    ds = gdal.Open(link)
    print('image opened')
    # Setting the bounds of the area of interest:
    xmin, xmax, ymin, ymax = 98400, 98930, 493058, 493990
    print('Translating...')
    test = gdal.Translate(destName=create_file_name(link),
                srcDS=ds,
                options=gdal.TranslateOptions(projWin = [xmin, ymax, xmax, ymin]),
                                                creationOptions = ['TFW=YES', 'COMPRESS=LZW'])
    print('translated')
    # This triggers writing the file (for some reason...)
    test = None

In [7]:
for link in links_5cm[1:]:
    save_img(link)

opening image


In [15]:
for link in links_1m[0:1]:
    save_img(link)